# # text_to_3d.ipynb
---

In [ ]:
import os
import numpy as np

np

In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [4]:
batch_size = 1
guidance_scale = 15.0
prompt = "a plush toy of a corgi nurse"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=100,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
render_mode = 'nerf' # you can change this to 'stf'
size = 32 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhIAAgAIcAAMbe2sXd2bra1rfZ0r7W0rfQyLjOyK7PuLTKvLLIwq/Iv6zMsr…

In [7]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh
import os

name = prompt.replace(' ', '_')
os.makedirs(f'./output/{name}', exist_ok=True)

for i, latent in enumerate(latents):
    with open(f'./output/{name}/{name}_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)
        # decode_latent_mesh(xm, latent).write_ply(f) # color

True
